In [1]:
import geopandas as gpd
import pandas as pd
import os
import io
import requests
import boto3

# Load all features

In [8]:
# get all c4F cities boundaries at different levels
boundaries_cities_c4f = gpd.read_file('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-cities-cities4forests.geojson')
boundaries_cities_c4f.head()

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.40125 -12.96457, -38.40126..."
1,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.39898 -12.96269, -38.39898..."
2,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.37346 -12.93345, -38.37386..."
3,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.41741 -12.97578, -38.41746..."
4,BRA-Salvador_ADM4_5,ADM4,Jardim Armação,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.43383 -12.98742, -38.43386..."


In [2]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


# Create empty indicator data frame

In [9]:
# create the table
cities_indicators_df = pd.DataFrame(boundaries_cities_c4f.drop(columns=['geometry','creation_date']))
cities_indicators_df.head()

,geo_id,geo_level,geo_name,geo_parent_name
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador
1,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador
2,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador
3,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador
4,BRA-Salvador_ADM4_5,ADM4,Jardim Armação,BRA-Salvador


# Upload in aws s3

In [10]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [11]:
# upload to aws
key_data = 'data/indicators/cities_indicators.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_df.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [12]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')